In [64]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression


# First computing coefficients using Scikit-Learn's Linear Regression module

In [65]:
A = np.array([
    [1,1],
    [2,4],
    [4,4],
    [6,5]
])

In [66]:
b = np.array([6,8,9,11])

In [67]:
model = LinearRegression(fit_intercept=False)
model.fit(A, b)
beta_lr = model.coef_

print("\nCoefficients from Linear Regression:")
print(beta_lr)


Coefficients from Linear Regression:
[0.3451957295 1.896797153 ]


# computing coefficients using Normal Equations

### The equation for linear regression is Ax = b. However, most of the time, the number of observations are greater than the number of predictors, resulting in a non-square matrix, meaning that A is not invertible. Therefore we will not be able to do x = A^-1b.

### Instead we need to find the values of x such that the predicted values Ax are as close to y as possible. So, we use x = (A^TA)^-1A^Ty, which is a derived formula from minimizing the residual sum of squares

In [68]:
A = np.array([
    [1,1],
    [2,4],
    [4,4],
    [6,5]
])

A

array([[1, 1],
       [2, 4],
       [4, 4],
       [6, 5]])

In [69]:
np.linalg.inv(A.T @ A) @ (A.T @ b)

array([0.3451957295, 1.896797153 ])

# SVD

In [70]:
AT = A.T 

#this is the transpose of A

In [71]:
ATA = AT @ A 
#this computes A^T * A
ATA

array([[57, 55],
       [55, 58]])

# Find Eigenvalues for Sigma Matrix

In [72]:
np.poly(ATA) # this will return the coefficients of the characteristic polynomial of the matrix A^TA

array([   1., -115.,  281.])

det($A^TA$ - lambda*I) = lambda^2 - 115lambda + 281

In [73]:
lambda1 = (115 + np.sqrt(((-115)**2)-4*281)) / 2

# here, I use the quadratic formula to solve for lambda1

In [74]:
lambda2 = (115 - np.sqrt(((-115)**2)-4*281)) / 2
# here, I use the quadratic formula to solve for lambda2

In [75]:
lambda1, lambda2

(np.float64(112.50227268031749), np.float64(2.497727319682511))

Therefore, eigenvalues are 2.497727319682511 and 112.50227268031749

In [76]:

#after finding the eigenvalues, we need to take the square root to find the singular values

singular_value1 = np.sqrt(lambda1)
singular_value2 = np.sqrt(lambda2)
sing_vals = singular_value1, singular_value2
sing_vals

(np.float64(10.60670885243474), np.float64(1.5804199820561973))

In [77]:
sigma_matrix = np.zeros(A.shape)
np.fill_diagonal(sigma_matrix, sing_vals)
sigma_matrix

array([[10.6067088524,  0.          ],
       [ 0.          ,  1.5804199821],
       [ 0.          ,  0.          ],
       [ 0.          ,  0.          ]])

# Find eigenvectors for V matrix

In [78]:
#now to find the eigenvectors I need to solve ATA - lambda1*I

first = ATA - (lambda1 * np.eye(ATA.shape[0]))
first

array([[-55.5022726803,  55.          ],
       [ 55.          , -54.5022726803]])

In [79]:
# first, we will do [-55.50227268 55][x y]^T
# this comes out to -55.50227268x + 55y = 0
# -55.50227268x = -55y
# then solve for x -> x = -0.9909504123694091

first[0][1] / -first[0][0]

np.float64(0.9909504123694091)

In [80]:
v1 = np.array([0.9909504123694091,1])

#to confirm if this is right, do -55.50227268(0.9909504123694091) + 55(1) and it will equal 0

In [81]:
#now we need to normalize the eigenvector

v1 = v1 * 1/np.sqrt(np.float64(0.9909504123694091**2) + 1)
v1

array([0.7038854547, 0.7103134989])

In [82]:
# now we repeat the same process as above
second = ATA - (lambda2*np.eye(ATA.shape[0]))
second

array([[54.5022726803, 55.          ],
       [55.          , 55.5022726803]])

In [83]:
-second[0][1] / second[0][0]

np.float64(-1.0091322305512271)

In [84]:
v2 = np.array([-1.0091322305512271,1])

In [85]:
v2 = v2 * 1/np.sqrt(np.float64((-1.0091322305)**2) + 1)

In [86]:
v2

array([-0.7103134989,  0.7038854547])

In [87]:
V = np.column_stack((v1, v2))
VT = V.T

In [88]:
# the V matrix is:
V

array([[ 0.7038854547, -0.7103134989],
       [ 0.7103134989,  0.7038854547]])

# Find Matrix U. For this we need to find orthonormal vectors

In [89]:
# u1 = 1/sigma1 * Av1

u1= (1/singular_value1) * (A@v1)
u1

array([0.1333306093, 0.4005978635, 0.533322437 , 0.7330153331])

In [90]:
# u2 = 1/sigma2 * Av2

u2 = (1/singular_value2) * (A@v2)

In [91]:
u2

array([-0.0040673013,  0.8826228703, -0.0162692052, -0.4697825443])

### To find u3 and u4 we must use the Gram-Schmidt process to ensure orthogonality

In [92]:
e3 = np.array([0, 0, 1, 0]) # standard basis vector

u3 = e3 - (((u1@e3/u1@u1)*u1) + ((u2@e3/u2@u2)*u2))
u3 

array([-0.2846975089, -0.7971530249, -0.1387900356, -1.5943060498])

In [93]:
e4 = np.array([0, 0, 0, 1])

u4 = e4 - (((u1@e4/u1@u1)*u1) + ((u2@e4/u2@u2)*u2) + ((u3@e4/u3@u3)*u3))
u4 

array([ -2.2141563557,  -4.5996377958,  -2.4794012234, -12.1992755918])

In [94]:
np.set_printoptions(suppress=True, precision=10)
U = np.column_stack((u1, u2,u3,u4))
U

array([[  0.1333306093,  -0.0040673013,  -0.2846975089,  -2.2141563557],
       [  0.4005978635,   0.8826228703,  -0.7971530249,  -4.5996377958],
       [  0.533322437 ,  -0.0162692052,  -0.1387900356,  -2.4794012234],
       [  0.7330153331,  -0.4697825443,  -1.5943060498, -12.1992755918]])

In [97]:
A_reconstructed = U @ sigma_matrix @ VT
print("\nReconstructed Matrix:")
print(A_reconstructed)

#this closely matches the original matrix meaning we did it correctly


Reconstructed Matrix:
[[1.           1.          ]
 [1.9999999999 4.0000000001]
 [4.           4.          ]
 [6.           5.          ]]


In [102]:
#now to find the coefficients we need to use the formula:
# V * sigma_inv * U^T

sigma_inv = np.zeros((sigma_matrix.shape[1], sigma_matrix.shape[0]))

# Compute the pseudo-inverse for non-zero entries
non_zero_mask = sigma_matrix > 1e-10  # Threshold to consider values as non-zero
sigma_inv[non_zero_mask.T] = 1 / sigma_matrix[non_zero_mask]  # Apply reciprocal and transpose mask


In [103]:
sigma_inv

array([[0.0942799519, 0.          , 0.          , 0.          ],
       [0.          , 0.632743202 , 0.          , 0.          ]])

In [104]:
V@sigma_inv@U.T@b

array([0.3451957295, 1.8967971531])

In [110]:
np.array([0.3451957295, 1.8967971531])


array([0.3451957295, 1.8967971531])

# Let's confirm if we get the same coefficients using Scikit Learn's Linear Regression package

In [108]:
from sklearn.linear_model import LinearRegression


In [109]:
model = LinearRegression(fit_intercept=False)
model.fit(A, b)
beta_lr = model.coef_

print("\nCoefficients from Linear Regression:")
print(beta_lr)


Coefficients from Linear Regression:
[0.3451957295 1.896797153 ]


# As you can see, the coefficients are the same as above!